### 1. 라이브러리 불러오기

In [1]:
# 데이터 처리용 라이브러리
import pandas as pd
import numpy as np

In [2]:
import warnings

In [3]:
warnings.filterwarnings(action="ignore")

### 2. 데이터 불러오기

In [4]:
featureData = pd.read_csv("../dataset/kopo_decision_tree_all_new.csv")

In [5]:
featureData.shape

(32415, 12)

### 3. 머신러닝 적용성 체크

In [6]:
### 머신러닝을 50건 이상인 아이템에 대해서만 돌리겠다

In [7]:
groupKey = ["REGIONID","ITEM"]

In [8]:
groupData = featureData.groupby(by=groupKey)[["YEARWEEK"]].agg("count").reset_index()

In [9]:
groupData.rename(columns={"YEARWEEK":"YEARWEEK_CNT"}, inplace=True)

In [10]:
### 데이터 병합 (기존데이터 + 머신러닝 타겟별 카운트데이터)

In [11]:
mergedData = pd.merge(left=featureData,
         right=groupData,
         on= groupKey)

In [12]:
mlActive = 50

In [13]:
mlTargetData = mergedData.loc[mergedData.YEARWEEK_CNT > mlActive]

In [14]:
mergedData.shape

(32415, 13)

In [15]:
mergedData.drop_duplicates(groupKey).shape
mlTargetData.drop_duplicates(groupKey).shape
## 258

(258, 13)

### 4. 머신러닝 적용

In [16]:
mlTargetData = mlTargetData.reset_index(drop=True)

### 4-1. 특성선정

In [17]:
from sklearn.preprocessing import LabelEncoder

In [18]:
ynLe = LabelEncoder()

In [19]:
mlTargetData["HOLIDAY_YN"] = ynLe.fit_transform(mlTargetData.HOLIDAY)

In [20]:
yn2Le = LabelEncoder()

In [21]:
mlTargetData["PROMOTION_YN"] = yn2Le.fit_transform(mlTargetData.PROMOTION)

In [22]:
### 4-2. 특성선정 (GROUPBy)

In [23]:
def mlGroup(eachGroup):
    # print( eachGroup.loc[:,groupKey].head(1) )
    # groupData = mlTargetData.groupby(by=groupKey)
    # eachGroup = groupData.get_group( list(groupData.groups)[2])
    label = ["QTY"]

    corrDf = eachGroup.corr()

    stdCorr = 0.00001

    features = list( corrDf.loc[ (corrDf["QTY"] > stdCorr) & 
                (corrDf["QTY"] != 1) ].index )

    ### 4-3. 데이터 분리

    from sklearn.model_selection import train_test_split

    trainingDataFeatures,\
    testDataFeatures,\
    trainingDataLabel,\
    testDataLabel =\
    train_test_split(eachGroup.loc[:,features],
                     eachGroup.loc[:,label],
                     test_size=0.2)

    # print(trainingDataFeatures.shape)
    # print(testDataFeatures.shape)
    # print(trainingDataLabel.shape)
    # print(testDataLabel.shape)

    ### 4-4. 머신러닝 적용 및 훈련

    from sklearn.tree import DecisionTreeRegressor

    modelDt = DecisionTreeRegressor(random_state=10)

    modelDtFit = modelDt.fit(trainingDataFeatures, trainingDataLabel)

    ### 4-5. 예측

    predictValue = modelDtFit.predict(testDataFeatures)

    testIndex = testDataFeatures.index

    testDataAll = eachGroup.loc[testIndex]

    testDataAll["PREDICT_DT"] = predictValue

    return testDataAll

In [24]:
groupData = mlTargetData.groupby(by=groupKey).apply(mlGroup)

(71, 3)
(18, 3)
(71, 1)
(18, 1)
(74, 4)
(19, 4)
(74, 1)
(19, 1)
(71, 3)
(18, 3)
(71, 1)
(18, 1)
(76, 3)
(19, 3)
(76, 1)
(19, 1)
(104, 1)
(26, 1)
(104, 1)
(26, 1)
(111, 4)
(28, 4)
(111, 1)
(28, 1)
(112, 4)
(28, 4)
(112, 1)
(28, 1)
(111, 4)
(28, 4)
(111, 1)
(28, 1)
(74, 4)
(19, 4)
(74, 1)
(19, 1)
(93, 4)
(24, 4)
(93, 1)
(24, 1)
(97, 4)
(25, 4)
(97, 1)
(25, 1)
(49, 4)
(13, 4)
(49, 1)
(13, 1)
(50, 4)
(13, 4)
(50, 1)
(13, 1)
(49, 4)
(13, 4)
(49, 1)
(13, 1)
(50, 6)
(13, 6)
(50, 1)
(13, 1)
(49, 6)
(13, 6)
(49, 1)
(13, 1)
(88, 3)
(23, 3)
(88, 1)
(23, 1)
(60, 4)
(16, 4)
(60, 1)
(16, 1)
(72, 5)
(19, 5)
(72, 1)
(19, 1)
(116, 4)
(29, 4)
(116, 1)
(29, 1)
(116, 4)
(30, 4)
(116, 1)
(30, 1)
(116, 4)
(30, 4)
(116, 1)
(30, 1)
(58, 6)
(15, 6)
(58, 1)
(15, 1)
(64, 4)
(17, 4)
(64, 1)
(17, 1)
(59, 5)
(15, 5)
(59, 1)
(15, 1)
(44, 3)
(11, 3)
(44, 1)
(11, 1)
(54, 3)
(14, 3)
(54, 1)
(14, 1)
(40, 3)
(11, 3)
(40, 1)
(11, 1)
(116, 4)
(30, 4)
(116, 1)
(30, 1)
(88, 6)
(22, 6)
(88, 1)
(22, 1)
(116, 4)
(30, 4)
(116, 1